### Setup Redshift Connection

In [1]:
# !pip list

In [1]:
import pandas as pd
import psycopg2
import numpy as np

In [2]:
host_name = "aiwedstrijd-cluster.cnih13yun1pa.eu-central-1.redshift.amazonaws.com"
port_num = "5439"
db_name = "dev"
username = "aiwedstrijd-superuser"
password = "aiwedstrijd-admin-g8sjt0D#"

In [3]:
def get_conn():
    conn = psycopg2.connect(dbname=db_name,
                            user=username,
                            password=password,
                            port=port_num,
                            host=host_name)
    return conn

In [4]:
def get_df(query):
    #print("Reading from Redshift...")
    with get_conn() as conn:
        with conn.cursor() as cur:
            #print('Start query')
            cur.execute(query)
            result_set = cur.fetchall()
            #print('Query klaar - nu python nog')
            colnames = [desc.name for desc in cur.description]
            df = pd.DataFrame.from_records(result_set, columns=colnames)
    #print("Done reading from Redshift")
    return df

### Query JourneyPart and QuestionTulp

There are 6 tables to be found in your database. The main two tables log the process of a user making a travel-request (QuestionTulp) and the application giving an answer to this question (JourneyPart). 

The **QuestionTulp** table is relatively straight forward, as it contains 1 row per travel request. This row tells you the following:
- question_tulp_id : Key to link the request to the answer
- Request_date and Request_time: The date and time the user made his/her request. Pplease note request_time is in minutes after midnight, e.g. 648 means 10:48 AM. 
- Travel_date and travel_time: The date and time the user WISHES his/her journey begins. Could be a few minutes, hours or days after the time the request is made.
- from_halteclusternumber and to_halteclusternumber: Indicates the start and end location of the desired journey, mapped to the closest bus/tram/train stop. These can be linked using the **station** table (use clustercode as key), where more details about the station can be found.
- Alternatively, if from_halteclusternumber and to_halteclusternumber are empty, we have more detailed information about the locations. This information can be found in **questiontulp_coordinates** and is linked using Question_Tulp_ID.

The **JourneyPart** table is a bit more complicated. It contains one row per part of the trip, e.g. a trip that starts with walking to the station, then a single train ride, and the a bus ride to go to the final destination, has THREE rows in the database. They all have the same QuestionTulpID, which is also the key that can be linked to the original travel request. 

Each of the rows will have certain characteristics, such as travel time, modality code (train, bus, etc. ; can be linked to **modality**  table for more details), line number, transport operator (can also be linked through **transport_companies** table). In a nutshell:
- question_tulp_id : Key to link the request to the answer
- journeypart_sequence_number : numbered action of the travel advice, 1 is first leg, 2 is second leg, etc.
- transport_company_id : id of transport company, links to **transport_companies** table
- modality_code_id : id of the modality used (e.g. walking is 51, bus is 2). Links to **modality** table.
- line_no : line number of modality used. Can be empty (usually for walking legs, modality code 51).
- start_cluster_number and end_cluster_number : contain the station/stop codes where the leg starts and ends. Can be linked using the **station** table (clustercode is key). Can be empty, for example for walking legs.
- travel time : time traveled on the leg in minutes
- start_station_location and end_station_location : same as start_cluster_number, uses different key though.
- departure_time : string containing date and time of departure, also available in columns daparturedata (timestamp for date) and departuretime (integer for time, again in minutes after midnight).
- request_time : same as departure_time, but now for the time the request was made (usually earlier than departure_time). 


In [11]:
# Query the journeyparts (reisadvies-regels)
query_jp = f"select TOP 100000 * from misstats.journeypart where  " \
        f" departuredate = '2020-01-01' " 
df_jp = get_df(query_jp)
df_jp['uurblok'] = df_jp['departuretime'].apply(lambda x: int(np.floor(int(x)/60)))
# df_jp['uurblok'] = df_jp['departuretime'].apply(int)


df_jp

,id,question_tulp_id,journeypart_sequence_no,transport_company_id,line_no,modality_code_id,start_cluster_number,end_cluster_number,travel_time,start_stationlocation,end_stationlocation,line_planning_name,service_code,departure_time,departuredate,departuretime,requestdate,requesttime,question_iteration,uurblok
0,18868344,8495162D-B29C-4B29-BA83-7662A3A1BD1E,2,1,11626,25,amf,shl,41,20849,49284,00116,,01012020 551,2020-01-01,551,2020-01-03,723,None,9
1,23406689,4AEA85B8-C9DB-4267-860F-9596E0C588F4,4,1,2237,25,dt,laa,12,2849,11373,00022,,01012020 624,2020-01-01,624,2020-01-03,606,None,10
2,18960655,8FA47314-D6AA-473C-92E7-AA8B9261F3C5,2,1,8190,127,gn,hrn,7,19424,20998,00081,,01012020 1408,2020-01-01,1408,2020-01-03,908,None,23
3,18898640,372B5357-6169-404E-8316-3749F11BF049,2,1,5175,127,rtd,rlb,9,10972,24254,00051,,01012020 1196,2020-01-01,1196,2020-01-03,782,None,19
4,23483236,FB822A31-2FDD-4C77-A10D-D1AE94D5BC9B,4,1,5758,127,dmnz,shl,16,27337,49284,00057,,01012020 1081,2020-01-01,1081,2020-01-03,769,None,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,3966195,ACD206B5-94B4-47F6-9556-EFC7F3CB572F,1,24,C,4,9608022,9608155,15,68128,68178,0M008,,01012020 840,2020-01-01,840,2020-01-01,193,None,14
99996,3967437,F26BCAF0-8709-4A08-A5D4-4233DC0D21E8,2,22,N82,6,9500502,5713003,32,55323,29918,00282,,01012020 262,2020-01-01,262,2020-01-01,240,None,4
99997,3969849,FF03D0B3-0CBA-40C6-88B3-BD30722713FF,2,22,N89,6,9500502,9508204,7,55330,66216,00289,,01012020 380,2020-01-01,380,2020-01-01,349,None,6
99998,3972975,44F14404-34D2-4FD9-8F01-AB0B25272912,4,22,3,29,9509110,9507007,3,30452,14938,00003,,01012020 626,2020-01-01,626,2020-01-01,491,None,10


In [6]:
df_jp.dtypes

id                                 object
question_tulp_id                   object
journeypart_sequence_no             int64
transport_company_id                int64
line_no                            object
modality_code_id                   object
start_cluster_number               object
end_cluster_number                 object
travel_time                        object
start_stationlocation              object
end_stationlocation                object
line_planning_name                 object
service_code                       object
departure_time                     object
departuredate              datetime64[ns]
departuretime                      object
requestdate                datetime64[ns]
requesttime                        object
question_iteration                 object
uurblok                             int64
dtype: object

In [12]:
query_qt = f"select TOP 100000 * from misstats.questiontulp where request_date = '2020-01-01' "

df_qt = get_df(query_qt)
df_qt

,id,question_iis_id,request_date,travel_date,request_day_type_id,request_day_part_id,travel_day_type_id,travel_day_part_id,action_id,from_postal_code_id,...,from_address,to_address,from_halteclusternumber,from_halteclusternumberlist,to_halteclusternumber,to_halteclusternumberlist,via_halteclusternumber,via_halteclusternumberlist,no_of_changes,reason_no_advice
0,119,0,2020-01-01,2020-01-01,1,4,1,4,3,,...,,,1200210,"1200210,1200200,1200604,1200601,1200001,hrn,12...",1000525,"1000525,1000522,1000529,1000543,1000563,100051...",,,2,
1,148,0,2020-01-01,2020-01-01,1,4,1,4,12,,...,,,9506021,9506021,5700010,"5700010,9506054,9506067,9506062,9506063,950607...",,,1,
2,188,0,2020-01-01,2020-01-01,1,4,1,4,3,,...,,,tb,"tb,7300100,7200100,7300045,7300989,7300567,730...",bet,"bet,6420030,6420109,6420100,6420010,6420045,64...",,,1,
3,284,0,2020-01-01,2020-01-01,1,4,1,4,2,,...,,,,,,,,,1,
4,54,0,2020-01-01,2020-01-01,1,4,1,1,19,,...,,,,,,,,,0,x
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,801089,0,2020-01-01,2020-01-01,1,4,1,1,4,,...,,,vl,"vl,6900001,6900002,6900806,6900201,6900808,690...",es,"es,4300001,4502106,4300003,4300545,4300005,430...",,,4,
99996,801185,0,2020-01-01,2020-01-01,1,4,1,4,13,,...,,,asd,"asd,9505021,9500502,9509500,9508033,9509910,95...",5633268,"5633268,5633068,5633266,5633257,5633308,563324...",,,6,
99997,801281,0,2020-01-01,2020-01-01,1,4,1,4,4,,...,,,5620005,"5620005,5620001,5620041,5620453,5620065,562000...",gvc,"gvc,9226005,9208732,9226001,9226009,9202096,54...",,,4,
99998,802097,0,2020-01-01,2020-01-02,1,4,1,2,4,,...,,,9508049,"9508049,9508045,9508051,9508097,9508043,950825...",rtd,"rtd,9608000,9601010,9601016,9601421,9601015,96...",,,5,


In [9]:
df_qt.dtypes

id                              int64
question_iis_id                 int64
request_date                   object
travel_date                    object
request_day_type_id             int64
request_day_part_id             int64
travel_day_type_id              int64
travel_day_part_id              int64
action_id                       int64
from_postal_code_id            object
to_postal_code_id              object
transfer_time_option            int64
question_type                   int64
planner_id                      int64
client_api_token_id            object
question_tulp_id               object
request_time                    int64
travel_time                     int64
via_coordinates                object
via_postalcode                 object
via_address                    object
from_address                   object
to_address                     object
from_halteclusternumber        object
from_halteclusternumberlist    object
to_halteclusternumber          object
to_halteclus

In [13]:
# Query the questiontulp (reisverzoeken)
# query_qt = f"select TOP 10 * from misstats.questiontulp where  " \
#         f" request_date = '2020-01-01 and modality_code_id = 1' "
# df_qt = get_df(query_qt)


In [13]:
#Load the necessary columns of the reisverzoeken
# basically this dataset is a subset of the Question Tulp, so the request_date is the same as previous one

df_reisverzoeken = df_qt[['id', 'question_iis_id', 'request_date', 'travel_date',
       'transfer_time_option', 'question_type', 'planner_id',
       'client_api_token_id', 'question_tulp_id', 'request_time',
       'travel_time', 'to_address', 'from_halteclusternumber',
        'to_halteclusternumber', 'no_of_changes', ]]
df_reisverzoeken.columns

Index(['id', 'question_iis_id', 'request_date', 'travel_date',
       'transfer_time_option', 'question_type', 'planner_id',
       'client_api_token_id', 'question_tulp_id', 'request_time',
       'travel_time', 'to_address', 'from_halteclusternumber',
       'to_halteclusternumber', 'no_of_changes'],
      dtype='object')

In [14]:
df_reisverzoeken

,id,question_iis_id,request_date,travel_date,transfer_time_option,question_type,planner_id,client_api_token_id,question_tulp_id,request_time,travel_time,to_address,from_halteclusternumber,to_halteclusternumber,no_of_changes
0,119,0,2020-01-01,2020-01-01,0,2,1,,C8AF4929-973C-43BA-B872-99E4070808B3,5,1445,,1200210,1000525,2
1,148,0,2020-01-01,2020-01-01,0,2,1,,0A20BB52-ED23-4E34-905E-82820BD78011,6,1445,,9506021,5700010,1
2,188,0,2020-01-01,2020-01-01,0,2,1,,B461A203-14F0-497E-A7F6-2DA437D6F0BC,7,1447,,tb,bet,1
3,284,0,2020-01-01,2020-01-01,0,2,2,,E76817BE-A008-45F6-9382-49B425642D54,11,1451,,,,1
4,54,0,2020-01-01,2020-01-01,0,2,1,,35E6C964-EF06-47F2-BB5E-6C885C33D93C,3,540,,,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,801089,0,2020-01-01,2020-01-01,0,2,3,qlKCXdgG8ZhjhvZAnZPj,C4DC020F-E6C7-4037-A34C-3DEF58279F6A,285,504,,vl,es,4
99996,801185,0,2020-01-01,2020-01-01,0,2,3,qlKCXdgG8ZhjhvZAnZPj,03983D9C-EC13-4C82-961C-BDD2019CD7A8,290,290,,asd,5633268,6
99997,801281,0,2020-01-01,2020-01-01,0,2,3,qlKCXdgG8ZhjhvZAnZPj,08B6783D-F474-4EB4-8CB0-81B3B36FDD28,293,292,,5620005,gvc,4
99998,802097,0,2020-01-01,2020-01-02,0,2,3,DyxHzhdgAV4JYJUcYXDN,8F5E1BB8-6B3A-477C-BA3D-3E2BFF2B34F8,328,793,,9508049,rtd,5


In [29]:
#Find the most frequent requested route

# Combine the two colu
combined_columns= df_reisverzoeken['from_halteclusternumber'].astype(str) + '-' + df_reisverzoeken['to_halteclusternumber']

# Use value_counts to count occurrences of each combination
combination_counts = combined_columns.value_counts()
print(combination_counts)
# Find the most frequent combination
most_frequent_combination = combination_counts.idxmax()

third_most_frequent_combination = combination_counts.index[2]

print("Most Frequent Combination:", third_most_frequent_combination)

rows_with_third_most_frequent_combination = df_reisverzoeken[combined_columns == third_most_frequent_combination]
print(rows_with_third_most_frequent_combination)

merged_df = pd.merge(rows_with_third_most_frequent_combination, df_jp, on='question_tulp_id', how='inner')
print(merged_df)

-                  59339
9608004-             114
6555018-ehv           83
1000101-              83
9508049-9601016       81
                   ...  
9508110-5714278        1
ut-9103004             1
6000437-4012482        1
5521024-gvc            1
9500559-dron           1
Name: count, Length: 26370, dtype: int64
Most Frequent Combination: 6555018-ehv
           id  question_iis_id request_date travel_date  transfer_time_option  \
101       294                0   2020-01-01  2020-01-01                     0   
403      8030                0   2020-01-01  2020-01-01                     0   
767     11010                0   2020-01-01  2020-01-01                     0   
2355    69900                0   2020-01-01  2020-01-01                     0   
3065    21479                0   2020-01-01  2020-01-01                     0   
...       ...              ...          ...         ...                   ...   
88067  617662                0   2020-01-01  2020-01-01                     0 

In [27]:
print(df_stations[df_stations['clustercode'] == "9508049"]['name'])
print(df_stations[df_stations['clustercode'] == "9601016"]['name'])

7502     Alexanderplein
24797    Alexanderplein
Name: name, dtype: object
12262    Rotterdam Centraal perron D
13307    Rotterdam Centraal perron B
Name: name, dtype: object


### Get Dimension tables

There are four dimension tables to consider:
- **Modality** : Links to modality_code_id in JourneyPart table
- **Transport_Companies** : Links to transport_company_id in JourneyPart table
- **Station** : Links to from_halteclusternumber and to_halteclusternumber in QuestionTulp table, and start_cluster_number and end_cluster_number in JourneyPart table
- **Questiontulp_Coordinates** links to Question_Tulp_ID in QuestionTulp table, shows the desired start and end coordinates of the journey, measured in Rijksdriehoek CRS. Is empty if from_halteclusternumber and to_halteclusternumber in QuestionTulp table are filled (and vice versa)

### Transportation Types

In [18]:
query_modality = f"select  * from misstats.modality"    
df_modality = get_df(query_modality)
df_modality

,id,modalitycode,transporttype,description
0,2,B,Bus,Bus
1,3,D,Bus,Stadsbus
2,4,M,Metro,Metro
3,5,V,Veerboot,Veerdienst
4,6,HA,Bus,Nachtbus
...,...,...,...,...
491,265,RB,Bus,Rolstoelbus
492,266,SLT,Trein,Slaaptrein
493,267,VP,Veerboot,Veerpont
494,268,WTA,Veerboot,Watertaxi


### Transportation Company List

In [19]:
query_transport_companies = f"select  * from misstats.transport_companies"    
df_transport_companies = get_df(query_transport_companies)
df_transport_companies

,transportcompanyid,basecompanyid,companyabbreviation,description,partofcompanyno,dataownercode
0,1,1,NS,NS,1,NS
1,2,221,NSI,NS International,1,NS
2,60,60,Keolis,Keolis,60,KEOLIS
3,4,224,Euregio,DB,1,NS
4,5,5,arriva,Arriva,5,ARR
...,...,...,...,...,...,...
212,914,338,Twents,Twents,60,KEOLIS
213,210,339,Eu Sleeper,European Sleeper,1,NS
214,601,340,601,Haagse Hopper Benoordenhout,340,VBF
215,602,341,602,Flextaxi Zeeland,341,GVZ


### Stations Dataset

In [20]:
query_stations = f"select  * from misstats.station"    
df_stations = get_df(query_stations)
display(df_stations)


,stationlocationid,name,city,municipality,province,country,x,y,clustercode
0,1542,Brabantlaan,Vught,Vught,Noord-Brabant,Nederland,148059,405773,6211380
1,5943,Kastelenplein,Eindhoven,Eindhoven,Noord-Brabant,Nederland,158622,380999,6411669
2,10353,Aalsmeerplein,Amsterdam,Amsterdam,Noord-Holland,Nederland,118177,484477,9504102
3,14798,Bontekoestraat,Breda,Breda,Noord-Brabant,Nederland,111390,398804,7200781
4,19974,Leesten,Zutphen,Zutphen,Gelderland,Nederland,212439,460223,4150148
...,...,...,...,...,...,...,...,...,...
26913,10891,Kramatweg,null,null,null,null,124906,486563,null
26914,6454,Nijhofflaan/Groenekruislaan,null,null,null,null,107465,423507,null
26915,1318,Oud-Turnhout KUILTJESSTRAAT,null,null,null,null,126020,372531,null
26916,73332,P+R Gieten,Gieten,Aa en Hunze,Drenthe,Nederland,246509,558875,1220801


In [25]:
print(df_stations[df_stations['clustercode'] == "9508049"]['name'])

7502     Alexanderplein
24797    Alexanderplein
Name: name, dtype: object


In [21]:
query_coordinates = f"select TOP 1000 * from misstats.questiontulp_coordinates"    
df_coordinates = get_df(query_coordinates)
df_coordinates

,question_tulp_id,from_coordinates_x,from_coordinates_y,to_coordinates_x,to_coordinates_y
0,C628A2FC-3A32-4A3F-A9AA-88C43753EED1,211600,473900,228200,487000
1,C72C0C49-2C24-4AE4-8402-CF81E34D45FB,81000,316600,121900,488000
2,B51C35E5-BAEB-459D-8F34-FE49B4A36036,161400,383800,134000,396900
3,75342C02-F910-409D-BC9F-C9D338BB977F,None,None,95500,434300
4,09E97250-2BEE-4116-85C1-D368346713D3,None,None,96200,437400
...,...,...,...,...,...
995,9A07ABBC-52CA-4D03-A9AC-3D90797A717E,None,None,None,None
996,1E36AE7D-55FE-409D-A7E7-138159F76AD3,136000,455700,107600,427400
997,05DA3FC4-CE28-436E-8237-41E8288240E4,None,None,None,None
998,19DDFFAD-1486-44AF-9FFE-3115DBF22064,91900,437800,105400,424500
